In [1]:
# TensorFlow and tf.keras
import tensorflow as tf
import tensorflow_datasets as tfds

import time
import sys 
import os
import numpy as np
import pandas as pd
import gc

import tensorflow.keras.layers as layers

from tensorflow.python.client import device_lib

print(tf.__version__)

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

print(device_lib.list_local_devices())

2.6.0
Num GPUs Available:  0
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11424900049303371975
]
